In [ ]:
import pandas as pd

In [ ]:
# Cell Dimension: 633, 1149

# importing the map and droping the last empty column
df = pd.read_csv('maps/RDCmod3.txt', header=None)
df = df.drop(columns=[633])

# droping columns and rows with 0
df = df.loc[:, (df != 0).any(axis=0)]
df = df.loc[(df!=0).any(axis=1)]

# reverse the order of the DataFrame, to match the reality
df = df.iloc[::-1]
df.reset_index(inplace=True, drop=True)

#normalizing the dataframe
df_norm = -df.copy()
for column in df_norm.columns:
    df_norm[column] = df_norm[column] / df_norm[column].abs().max()
    
#Rounding the dataframe

#tester en gardant le vide dans la carte, soulage le simulateur ?
df_rounded = df_norm.round(0).astype(int)
df_rounded = df_rounded + 1
df_rounded.replace(2,1, inplace=True)

#Exporting the final map
df_rounded.to_csv("maps/mapnorm.csv",sep=';', header=False, index=False)

### Test en enlevant les murs derrière les murs

In [ ]:
dftest = pd.read_csv("maps/mapnorm.csv",sep=';', header=None)
dftest2 = dftest.copy()

In [ ]:
print(id(dftest))
print(id(dftest2))

In [ ]:
for y in range(len(dftest)):
    for x in range(len(dftest.iloc[y])):
        if not y==0 and not x==0 and not y==681 and not x==252:
            if(dftest.iloc[y-1][x] == 1 and dftest.iloc[y+1][x] == 1 and dftest.iloc[y][x-1] == 1 and dftest.iloc[y][x+1] == 1):
                dftest2.iloc[y][x] = 0
        else:
            dftest2.iloc[y][x] = 0

In [ ]:
dftest2.to_csv("maps/mapnorm1.csv",sep=';', header=False, index=False)

### Test avec 2D Convolution

In [ ]:
from scipy.signal import convolve2d
import numpy as np

a = df_rounded.values
print(a)
print(a.shape)


#on divise la taille de la carte par n
n = 2
kernel = np.ones((n, n))
convolved = convolve2d(a, kernel, mode='valid')
a_downsampled = convolved[::n, ::n] / n**2

print(a_downsampled)
print(a_downsampled.shape)

df_down = pd.DataFrame(a_downsampled)
df_down = df_down.round(0).astype(int)
df_down.to_csv("maps/mapdown.csv",sep=';', header=False, index=False)